<a href="https://colab.research.google.com/github/Ahtesham519/Genrative_Deep_learning_v2_2023/blob/main/Pixel_CNN_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PIXELCNN form SCRATCH


In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np

import tensorflow as tf
from tensorflow.keras import datasets, layers, models, optimizerz , callbacks

from notebooks.utils import display

ImportError: ignored

#0. Parameters

In [2]:
IMAGE_SIZE = 16
PIXEL_LEVELS = 4
N_FILTERS = 128
RESIDUAL_BLOCKS = 5
BATCH_SIZE = 128
EPOCHS = 150



#1. Prepare the data

In [3]:
#Load the data
(x_train, _), (_,_) = datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [4]:
#Prepare the data
def preprocess(imgs_int):
  imgs_int = np.expand_dims(imgs_int, -1)
  imgs_int = tf.image.resize(imgs_int, (IMAGE_SIZE, IMAGE_SIZE)).numpy()
  imgs_int = (imgs_int / (256 / PIXEL_LEVELS)).astype(int)
  imgs = imgs_int.astype("float32")
  return imgs, imgs_int

input_data , output_data = preprocess(x_train)

In [5]:
#Show some item of clothing from the training set
display(input_data)

array([[[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
         [1.],
         [2.],
         ...,
         [2.],
         [3.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]]],


       [[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [2.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [3.],
         [0.],
         [0.]],

        ...,

        [[0.],
 

#Build the PixelCNN

In [6]:
#The first layer is the PIXELCNN layer , This layer simply
#builds on the 2D convolutional layer, but includes masking

class MaskedConv2D(layers.Layer):
  def __init__(self, mask_type, **kwargs):
    super(MaskedConv2D, self).__init__()
    self.mask_type = mask_type
    self.conv = layers.Conv2D(**kwargs)

  def build(self, input_shape):
    #Build the conv2d layer to initialize kernel variables
    self.conv.build(input_shape)
    #Use the initialized kernel to create the mask
    kernel_shape = self.conv.kernel.get_Shape()
    self.mask = np.zeros(shape = kernel_shape)
    self.mask[:kernel_shape[0] // 2, ...] = 1.0
    self.mask[kernel_shape[0] // 2, kernel_shape[1] // 2, ...] = 1.0
    if self.mask_type == "B":
      self.mask[kernel_shape[0] // 2, kernel_shape[1] // 2, ...] = 1.0


  def call(self, inputs):
    self.conv.kernel.assign(self.conv.kernel * self.mask)
    return self.conv(inputs)


  def get_config(self):
    sfg = super().get_config()
    return cfg



In [7]:
class ResidualBlock(layers.Layer):
  def __init__(self, filters, **kwargs):
    super(ResidualBlock, self).__init__(**kwargs)
    self.conv1 = layers.Conv2D(
        filters = filters // 2, kernel_size = 1 , activation = "relu"
    )
    self.pizel_conv = MaskedConv2D(
        mask_type = "B",
        filters = filters // 2,
        kernel_size = 3,
        activation = "relu",
        padding = "same",

    )
    self.conv2 = layers.Conv2D(
        filters = filters, kernel_size = 1 , activation ="relu"
    )
  def call(self, inputs):
    x = self.conv1(inputs)
    x = self.pixel_conv2(x)
    x = self.conv2(x)
    return layers.add([inputs, x])

  def get_config(self):
    cfg = super().get_config()
    return cfg



In [8]:
inputs = layers.Input(shape = (IMAGE_SIZE, IMAGE_SIZE, 1))
x = MaskedConv2D(
    mask_type = "A",
    filters = N_FILTERS,
    kernel_size = 7,
    activation = "relu",
    padding = "same",
)(inputs)

for _ in range(RESIDUAL_BLOCKS):
  x = ResidualBlock(filters = N_FILTERS)(x)

for _ in range(2):
  x = MaskedConv2D(
      mask_type = "B",
      filters = N_FILTERS,
      kernel_size = 1,
      strides = 1,
      activation = "relu",
      padding = "valid",
  )(x)

out = layers.Conv2D(
    filters = PIXEL_LEVELS,
    kernel_size = 1,
    strides = 1,
    activation = "softmax",
    padding = "valid",
)(x)

pixel_cnn = model.Model(inputs, out)
pixel_cnn.summary()


AttributeError: ignored

#3. Train the PixelCNN


In [9]:
adam = optimizers.Adam(learning_rate = 0.0005)
pixel_cnn.compile(optimizers = adam , loss="sparse_categotical_crossentropy")

NameError: ignored

In [10]:
tensorboard_callback = callbacks.TensorBoard(log_dir = "./logs")

class ImageGenerator(callbacks.Callback):
  def __init__(self, num_img):
    self.num_img = num_img

  def sample_from(self, probs, temperature):
    probs = probs ** (1 / temperature)
    probs = probs / np.sum(probs)
    return np.random.choice(len(probs), p = probs)

  def genrate(self, temperature):
    genrated_images = np.zeros(
        shape = (self.num_img,) + (pixel_cnn.input_shape)[1:]
    )
    batch, rows , cols, channels = genrated_images.shape

    for row in range(rows):
      for col in range(cols):
        for channel in range(channels):
          probs =self.model.predict(genrated_images, verbose= 0)[
              :, row , col, :
          ]
          genrated_images[:, row , col, channel] = [
              self.sample_from(x , temperature) for x in probs
          ]
          genrated_images[:, row , col , channel] /= PIXEL_LEVELS

    return genrated_images


  def on_epoch_end(self, epoch , logs = None):
    generated_images = self.generate(temperature = 1.0)
    display(
        generated_images ,
        save_to = "./output/generated_img_%03d.png" % (epoch),
    )

img_generator_callback = ImageGenrator(num_img = 10)

NameError: ignored

In [11]:
pixel_cnn.fit(
    input_data,
    output_data,
    batch_size = BATCH_SIZE,
    epochs = EPOCHS,
    callbacks = [tensorboard_callbcak, img_generator_callback],

)

NameError: ignored

#4. Generate images

In [12]:
generated_images = img_generator_callback.generate(temperature = 1.0)

NameError: ignored

In [ ]:
display(generated_images)